In [1]:
import itertools
import os
import random
from collections import Counter, defaultdict
from copy import deepcopy
from dataclasses import asdict
from pathlib import Path

from termcolor import colored, cprint

from poe_types import *
from utils import loadRecombsFromFileList

In [2]:
# Load historical recombs for easier item bases and recomb data
json_dir = Path().parent / 'data/json'
recombination_files = sorted(os.listdir(json_dir))
recombination_files_full = [json_dir / name for name in recombination_files]
recombs = loadRecombsFromFileList(recombination_files_full)

In [3]:
# Calculate pool size before_after
before_after = defaultdict(list)
for fpath in recombination_files:
    data = recombs[fpath]
    countSlots = lambda item, slot: sum([m.getSlot() == slot for m in item.mods])
    input_prefix_count = countSlots(data['input1'], 'Prefix') + countSlots(data['input2'], 'Prefix')
    input_suffix_count = countSlots(data['input1'], 'Suffix') + countSlots(data['input2'], 'Suffix')
    output_prefix_count = countSlots(data['output'], 'Prefix')
    output_suffix_count = countSlots(data['output'], 'Suffix')
    
    before_after[input_prefix_count].append(output_prefix_count)
    before_after[input_suffix_count].append(output_suffix_count)

# Put it in a frequency version to make it simpler
bafreq = {}
for in_pool in sorted(list(before_after.keys())):
    if in_pool == 0: # Pretty obvious what happens here
        continue
    
    freq = Counter(before_after[in_pool])
    total = freq.total()
    bafreq[in_pool] = {outcome: round(num/total,  3) for outcome, num in freq.items()}

display(bafreq)

{1: {1: 0.583, 0: 0.417},
 2: {1: 0.744, 2: 0.232, 0: 0.024},
 3: {1: 0.383, 2: 0.574, 3: 0.043},
 4: {2: 0.529, 3: 0.355, 1: 0.116},
 5: {3: 0.557, 2: 0.443},
 6: {2: 0.26, 3: 0.74}}

In [4]:
# Non recomb helper crafting methods
C_PER_D = 185
COST_YELLOWBEAST_D = 1 / 48 # Redtarget000 pricing
COST_FENUMAL_PLAGUED_C = 8
COST_ANNUL_C = 2


def check_splitItem(item):
    explicit_mods = item.getPrefixes() + item.getSuffixes()
    if len(explicit_mods) <= 1:
        return (False, f'Provides no value')
    if len(explicit_mods) <= len(item.valuable_mod_indices):
        return (False, f'Will make item less valuable')
    if len(valuable_mod_indices) == 0:
        return (False, f'Will not concentrate any valuable mods')
    if item.special_types != []:
        return (False, f'Cannot split item with special_types {item.special_types}')
    
    return (True, '')


def splitItem(item):
    expected_cost_c = COST_YELLOWBEAST_D * 3 / C_PER_D + COST_FENUMAL_PLAGUED_C

    # Since I don't have data here, I assume splitting:
    # 1. produces two items which are centered around (mod count)/2, eg 5 -> 2 + 3
    # 2. does not care about prefix/suffix balancing
    # 3. produces magic item when explicit mods < 3, otherwise rare

    explicit_mods = item.getPrefixes() + item.getSuffixes()
    a_modcount = len(explicit_mods) // 2

    all_indices = set(range(len(explicit_mods)))
    output_indices = list(itertools.combinations(range(len(explicit_mods)), a_modcount))
    a_indices = output_indices[:len(output_indices) // 2]
    b_indices = output_indices[len(output_indices) // 2:][::-1]

    # Only want to keep items with valuable mods, rest get dumped

    print(PoEItem(**asdict(item)))
    
    print(a_indices)
    print(b_indices)

    # TODO: Implement


def check_annulItem(item):
    # TODO: Fix for new valuable implementation
    explicit_mods = item.getPrefixes() + item.getSuffixes()
    if len(explicit_mods) == 0:
        return (False, 'No explicit mods to annul')
    if len(explicit_mods) == len(item.valuable_mod_indices):
        return (False, 'Will annul valuable mod')
    return (True, '')


def annulItem(item):
    expected_cost_c = COST_ANNUL_C

    prefix_mods = [m for m in item.mods if m.getSlot() == 'Prefix']
    explicit_mods = [(i, m) for i, m in enumerate(item.mods) if m.getSlot() in ['Prefix', 'Suffix']]
    smited_mod_index = random.randint(0, len(explicit_mods)-1)
    item.mods.pop(smited_mod_index)

    return item

In [5]:
# List known one-handed axe modifiers
known_axe_modifiers = {'Prefix': set(), 'Suffix': set()}
for fpath in recombination_files:
    data = recombs[fpath]

    for item_type in ['input1', 'input2']:
        item = data[item_type]
        if item.iclass == 'One Hand Axes':
            for m in item.getAffixes():
                known_axe_modifiers[m.getSlot()].add('\n'.join([e.description for e in m.effects]))

for pool_type in ['Prefix', 'Suffix']:
    known_axe_modifiers[pool_type] = sorted(list(known_axe_modifiers[pool_type]))
for pool_type, mods in known_axe_modifiers.items():
    cprint(pool_type, attrs=['bold'])
    for m in mods:
        for line in m.split('\n'):
            print('   ', line)
        # print('    |')

Prefix
    +X to Level of Socketed Gems
    +X to Level of Socketed Melee Gems
    +X to maximum Mana (crafted)
    +X% to Damage over Time Multiplier for Bleeding from Hits with this Weapon
    +X% to Damage over Time Multiplier for Poison inflicted with this Weapon
    Adds X to X Chaos Damage
    Adds X to X Cold Damage
    Adds X to X Fire Damage
    Adds X to X Lightning Damage
    Adds X to X Physical Damage
    Adds X to X Physical Damage (crafted)
    Attacks with this Weapon Penetrate X% Elemental Resistances
    Socketed Gems are Supported by Level X Brutality — Unscalable Value
    X% increased Physical Damage
    Socketed Gems are Supported by Level X Melee Physical Damage — Unscalable Value
    X% increased Physical Damage
    Socketed Gems are Supported by Level X Ruthless — Unscalable Value
    X% increased Physical Damage
    X% increased Damage over Time (crafted)
    X% increased Elemental Damage with Attack Skills
    X% increased Physical Damage
    X% increased Phy

In [10]:
# Recomb crafting methods

junk_prefix_dict = {
    'category': 'Prefix',
    'title': '',
    'tier': -1,
    'tags': [],
    'effects': [PoEEffect(**
        {
            'actual_stats': [],
            'ranges': [],
            'description': 'Junk Prefix',
            'comment_lines': [],
        }
    )],
}
junk_suffix_dict = deepcopy(junk_prefix_dict)
junk_suffix_dict['category'] = 'Suffix'
junk_suffix_dict['effects'][0].description = 'Junk Suffix'


def check_recombineItems(item1, item2, valuable_mods):
    if len(valuable_mods) == 0:
        return (False, 'No valuable mods so no point recombining')    
    # if len(item1.valuable_mod_indices) == 0 or len(item2.valuable_mod_indices) == 0:
    #     # Technically this could work as a worse Fenumal Plagued Arachnid, maybe should turn off later
    #     return (False, 'No valuables on left or right')
    return (True, '')


def recombineItems(item1, item2, valuable_mods):
    input_pools = {
        'Prefix': item1.getPrefixes() + item2.getPrefixes(),
        'Suffix': item1.getSuffixes() + item2.getSuffixes(),
    }

    # Assume no weighting, doubling, modgrouping, or influence requirements for v1 to make things simpler
    # TODO: Make a separate before_after table for doubled and non doubled
    # TODO: Average ilvl
    
    # Generate outcomes for individual pools
    pool_output_mod_chances = defaultdict(list)
    for pool_type, mod_pool in input_pools.items():
        for outcome, pc in bafreq[len(mod_pool)].items():
            N = outcome
            possible_mod_combos = list(itertools.combinations(range(len(mod_pool)), N))
            pool_output_mod_chances[pool_type].extend(
                [(pc / len(possible_mod_combos), x) for x in possible_mod_combos]
            )
    # display(pool_output_mod_chances)
    
    # Combine modpools into final output item
    final_output_mod_chances = []
    for prefix_outcome in pool_output_mod_chances['Prefix']:
        for suffix_outcome in pool_output_mod_chances['Suffix']:
            ppc, prefix_pool = prefix_outcome
            spc, suffix_pool = suffix_outcome
            final_output_mod_chances.append((ppc * spc, (prefix_pool, suffix_pool)))

    # display(final_output_mod_chances)

    # Pick each base for every output chance (so doubling the number of output states)
    item_output_chances = []
    for pc, mod_outcome in final_output_mod_chances:
        for i in range(0, 2):
            item_output_chances.append((i, pc/2, mod_outcome))

    # display(item_output_chances[:10])
    # print(len(item_output_chances))
    
    # The final output chances is a huge list (eg ~280 for (2,2) + (1,3)), so simplify down to "valuable" and "junk" modifiers
    # If "valuable" modifiers drop below a certain tier, they are considered junk now
    # TODO: This may not work after doubling is implemented (if doubling doesn't naturally happen)

    # Get valuable indices for each pool
    valuable_indices = defaultdict(set)
    for pool_type, mod_pool in input_pools.items():
        for i, m in enumerate(mod_pool):
            for vm in valuable_mods:
                if m.stringDescription() == vm.description and m.tier <= vm.min_tier:
                    valuable_indices[pool_type].add(i)

    # Convert states to final mod + placeholder junk mods - hopefully this reduces the number of states to consider
    output_to_percent = Counter()
    for item_base_index, output_prob, mod_pair_indices in item_output_chances:
        output_prefix_pool, output_suffix_pool = mod_pair_indices

        compressed_prefix_pool = tuple(
            sorted((
                (input_pools['Prefix'][m] if m in valuable_indices['Prefix'] else PoEMod(**junk_prefix_dict))
                for m in output_prefix_pool
            ), key=lambda mod: mod.stringDescription())
        )
        compressed_suffix_pool = tuple(
            sorted((
                (input_pools['Suffix'][m] if m in valuable_indices['Suffix'] else PoEMod(**junk_suffix_dict))
                for m in output_suffix_pool
            ), key=lambda mod: mod.stringDescription())
        )
        
        compressed_state = (
            item_base_index,
            compressed_prefix_pool,
            compressed_suffix_pool,
        )
        output_to_percent[compressed_state] += output_prob

    return output_to_percent


def pprintRecombinatorOutcomes(output_to_percent, valuable_inputs, compression_level = 1):
    # print('Relevant outcomes:', len(output_to_percent))
    
    # For now, compress different bases into a single item
    user_outcomes = Counter()
    for state, percent in list(output_to_percent.items()):
        ps_short = (len(state[1]), len(state[2]))
        user_state = (
            ps_short,
            tuple(sorted([m.stringDescription() for m in state[1] if not m.stringDescription().startswith('Junk')])),
            tuple(sorted([m.stringDescription() for m in state[2] if not m.stringDescription().startswith('Junk')])),
        )

        if compression_level == 0:
            print(f'{round(percent*100, 2)}% {ps_short}')
            for pool_idx in range(1, 3):
                for m in state[pool_idx]:
                    print(m.stringDescription())
            print()
        
        user_outcomes[user_state] += percent


    level2_outcomes = Counter()
    level3_prefix_suffix = {}
    for outcome, percent in sorted(user_outcomes.items(), key=lambda x: x[1], reverse=True):
        ps_short = outcome[0]
        
        valuables = len(outcome[1]) + len(outcome[2])
        valuable_score = valuables - max(valuable_inputs) # Number of mods lost relative to parent inputs
        if valuable_score < 0 or valuables == 0:
            goodness = 'red'
        if valuable_score == 0:
            goodness = 'yellow'
        if valuable_score > 0:
            goodness = 'green'

        if goodness not in level3_prefix_suffix:
            level3_prefix_suffix[goodness] = defaultdict(float)
        level3_prefix_suffix[goodness][ps_short] += percent

        level2_state = (
            goodness,
            *outcome[1:]
        )
        level2_outcomes[level2_state] += percent
        
        if compression_level == 1:
            cprint(f'{round(percent*100, 2)}% {ps_short}', goodness)
            for sm in outcome[1]:
                print(f'(Prefix) {sm}')
            for sm in outcome[2]:
                print(f'(Suffix) {sm}')
            print()
    
    level3_outcomes = Counter()
    for outcome, percent in sorted(level2_outcomes.items(), key=lambda x: x[1], reverse=True):
        goodness = outcome[0]
        valuables = len(outcome[1]) + len(outcome[2])

        if valuables == 0:
            level3_outcomes['BRICK'] += percent
        else:
            level3_outcomes[goodness] += percent
        
        if compression_level == 2:
            cprint(f'{round(percent*100, 2)}%', goodness)
            for sm in outcome[1]:
                print(f'(Prefix) {sm}')
            for sm in outcome[2]:
                print(f'(Suffix) {sm}')
            if valuables == 0:
                cprint('BRICK', 'red', attrs=['bold'])
            print()

    if compression_level == 3:
        messages = {
            'red': 'Lose mods',
            'yellow': 'Stay max mods',
            'green': 'Gain mods',
            'BRICK': 'BRICK',
        }
        for goodness, percent in sorted(level3_outcomes.items(), key=lambda x: x[1], reverse=True):
            pcstr = round(percent * 100, 1)

            color = goodness
            if goodness == 'BRICK':
                color = 'red'
            cprint(f'{messages[goodness]}: {pcstr}%', color)
            
            if goodness != 'BRICK':
                for ps_short, ps_percent in sorted(level3_prefix_suffix[goodness].items(), key=lambda x: x[1], reverse=True):
                    print('   ', ps_short, f'{str(round(ps_percent * 100, 1)).rjust(5)}%')

        # display(level3_prefix_suffix)


@dataclass
class ValuableMod:
    description: str
    min_tier: int


CHOSEN_ITEM_1 = deepcopy(recombs['00251.json']['input1'])
CHOSEN_ITEM_2 = deepcopy(recombs['00251.json']['input2'])
valuable_mods = [
    ValuableMod('X% increased Physical Damage|+X to Accuracy Rating', 4),
    ValuableMod('X% increased Physical Damage', 3),
    ValuableMod('Socketed Gems are supported by Level X Multistrike — Unscalable Value|X% increased Attack Speed', 2),
    ValuableMod('X% increased Physical Damage|Hits with this Weapon have Culling Strike against Bleeding Enemies — Unscalable Value', 1),
    ValuableMod('+X% to Damage over Time Multiplier for Bleeding from Hits with this Weapon', 2)
]
msg = check_recombineItems(CHOSEN_ITEM_1, CHOSEN_ITEM_2, valuable_mods)
if msg[0]:
    output_to_percent = recombineItems(CHOSEN_ITEM_1, CHOSEN_ITEM_2, valuable_mods)
else:
    print(msg[1])

pprintRecombinatorOutcomes(output_to_percent, (3, 2), compression_level=3)
# print(CHOSEN_ITEM)

Lose mods: 46.5%
    (2, 2)  14.3%
    (3, 2)  13.6%
    (2, 3)  11.1%
    (3, 3)   8.4%
Stay max mods: 38.2%
    (3, 3)  14.3%
    (2, 3)  10.4%
    (3, 2)   8.9%
    (2, 2)   4.7%
Gain mods: 14.4%
    (3, 3)   8.4%
    (2, 3)   3.2%
    (3, 2)   2.2%
    (2, 2)   0.6%
BRICK: 0.8%


In [7]:
# Some craftofexile manual crafting simulator stuff
# All these assume starting with an ilvl86 elder reaver axe

COST_T1_PHYS_ESSENCE_C = 6
COST_JAGGED_FOSSIL_C = 1.6
COST_SERRATED_FOSSIL_C = 3
COST_1_RESONATOR_C = 2
COST_VIVID_C = 1/35
COST_WILD_C = 1/60
COST_ALTERATION_C = 1/7
COST_ALCH_C = 1/25
COST_SCOUR_C = 1/5


@dataclass
class ReaverAxeOutcomes:
    strategy: str
    cost_c: float
    simulated: int
    hits: int
    essence_added_phys: int
    culling: int
    t1bleeddmg: int
    t1atkspd: int
    t1phys: int
    t1hybrid: int

@dataclass
class SquireAxeOutcomes:
    strategy: str
    cost_c: float
    simulated: int
    hits: int
    t2_phys_plus_t2_added: int
    culling: int
    gem_20_bleed: int
    gem_20_multistrike: int
    gem_20_melee_phys: int

In [67]:
# List of COE outputs
strategies = [
    ReaverAxeOutcomes(
        'T1 Contempt Essence',
        COST_T1_PHYS_ESSENCE_C,
        simulated=7660,
        hits=400,
        essence_added_phys=400,
        culling=178,
        t1bleeddmg=71,
        t1atkspd=139,
        t1phys=3,
        t1hybrid=6
    ),
    ReaverAxeOutcomes(
        'Jagged Fossil',
        COST_JAGGED_FOSSIL_C + COST_1_RESONATOR_C,
        1876,
        400,
        0,
        275,
        114,
        32,
        2,
        0,
    ),
    ReaverAxeOutcomes(
        'Serrated Fossil',
        COST_SERRATED_FOSSIL_C + COST_1_RESONATOR_C,
        3381,
        400,
        0,
        158,
        86,
        161,
        3,
        4,
    ),
    ReaverAxeOutcomes(
        'Harvest Physical Reforge',
        COST_VIVID_C * 50,
        3380,
        400,
        0,
        203,
        125,
        77,
        9,
        9,
    ),
    ReaverAxeOutcomes(
        'Harvest Speed Reforge',
        COST_VIVID_C * 150,
        simulated=2886,
        hits=400,
        essence_added_phys=0,
        culling=60,
        t1bleeddmg=49,
        t1atkspd=298,
        t1phys=2,
        t1hybrid=3,
    ),
    ReaverAxeOutcomes(
        'Harvest Attack Reforge',
        COST_WILD_C * 75,
        simulated=4732,
        hits=400,
        essence_added_phys=0,
        culling=166,
        t1bleeddmg=88,
        t1atkspd=144,
        t1phys=3,
        t1hybrid=4,
    ),
    # ReaverAxeOutcomes(
    #     'Alt Spamming',
    #     COST_ALTERATION_C,
    #     simulated=17044,
    #     hits=400,
    #     essence_added_phys=0,
    #     culling=158,
    #     t1bleeddmg=112,
    #     t1atkspd=127,
    #     t1phys=0,
    #     t1hybrid=3,
    # ),
    # ReaverAxeOutcomes(
    #     'Alch Scour Spamming',
    #     COST_ALCH_C + COST_SCOUR_C,
    #     simulated=5910,
    #     hits=400,
    #     essence_added_phys=0,
    #     culling=158,
    #     t1bleeddmg=92,
    #     t1atkspd=151,
    #     t1phys=7,
    #     t1hybrid=3,
    # ),
]

# strategies = [
#     SquireAxeOutcomes(
#         'Harvest Speed Reforge',
#         cost_c = COST_VIVID_C * 150,
#         simulated=2902,
#         hits=400,
#         t2_phys_plus_t2_added=0,
#         culling=61,
#         gem_20_bleed=57,
#         gem_20_multistrike=294,
#         gem_20_melee_phys=1,
#     ),
#     SquireAxeOutcomes(
#         'Harvest Attack Reforge',
#         cost_c = COST_WILD_C * 75,
#         simulated=4302,
#         hits=400,
#         t2_phys_plus_t2_added=0,
#         culling=141,
#         gem_20_bleed=133,
#         gem_20_multistrike=0,
#         gem_20_melee_phys=0,
#     ),
#     SquireAxeOutcomes(
#         'Harvest Physical Reforge',
#         cost_c = COST_VIVID_C * 50,
#         simulated=3018,
#         hits=400,
#         t2_phys_plus_t2_added=0,
#         culling=162,
#         gem_20_bleed=165,
#         gem_20_multistrike=31,
#         gem_20_melee_phys=0,
#     ),
#     SquireAxeOutcomes(
#         'T1 Contempt Essence',
#         cost_c = COST_T1_PHYS_ESSENCE_C,
#         simulated=5464,
#         hits=400,
#         t2_phys_plus_t2_added=16,
#         culling=145,
#         gem_20_bleed=126,
#         gem_20_multistrike=55,
#         gem_20_melee_phys=3,
#     ),
# ]

In [32]:
# Now print best methods
modifier_keys = asdict(strategies[0]).keys()
exclude = {'strategy', 'cost_c', 'simulated', 'hits'}
modifiers_keys = [k for k in modifier_keys if k not in exclude]
chaosPrint = lambda c: colored(f'{round(c, 1)}c', 'yellow')
mk_costs = {}
PRINT_INDIVIDUAL_STRATS = True
for strategy in strategies:
    if PRINT_INDIVIDUAL_STRATS:
        cprint(f'{strategy.strategy}', attrs=['bold'])
    cost_per_hit = strategy.cost_c * strategy.simulated / strategy.hits
    if PRINT_INDIVIDUAL_STRATS:
        print(f'    {chaosPrint(cost_per_hit)} per hit')
    for mk in modifiers_keys:
        freq = getattr(strategy, mk)
        if freq == 0:
            cost_per_modifier_hit = 1000000000000000000000000000000000000000000000
            if PRINT_INDIVIDUAL_STRATS:
                print(f'    {colored("∞", "yellow")} per {mk}')
        else:
            cost_per_modifier_hit = cost_per_hit * strategy.hits / freq
            if PRINT_INDIVIDUAL_STRATS:
                print(f'    {chaosPrint(cost_per_modifier_hit)} per {mk}')

        if mk in mk_costs:
            if cost_per_modifier_hit < mk_costs[mk][1]:
                mk_costs[mk] = (strategy.strategy, cost_per_modifier_hit)
        else:
            mk_costs[mk] = (strategy.strategy, cost_per_modifier_hit)

cprint('Cheapest methods:', attrs=['bold'])
# mk_costs['essence_added_phys'] = ('T1 Contempt Essence', COST_T1_PHYS_ESSENCE_C) # For clarity
for mk, cheapest in mk_costs.items():
    strat, cost = cheapest
    print(f'    {mk.ljust(len("essence_added_phys"))}: {chaosPrint(cost)} using "{strat}"')

# TODO: Need to calculate average junk modifiers

T1 Contempt Essence
    114.9c per hit
    114.9c per essence_added_phys
    258.2c per culling
    647.3c per t1bleeddmg
    330.6c per t1atkspd
    15320.0c per t1phys
    7660.0c per t1hybrid
Jagged Fossil
    16.9c per hit
    ∞ per essence_added_phys
    24.6c per culling
    59.2c per t1bleeddmg
    211.1c per t1atkspd
    3376.8c per t1phys
    ∞ per t1hybrid
Serrated Fossil
    42.3c per hit
    ∞ per essence_added_phys
    107.0c per culling
    196.6c per t1bleeddmg
    105.0c per t1atkspd
    5635.0c per t1phys
    4226.2c per t1hybrid
Harvest Physical Reforge
    12.1c per hit
    ∞ per essence_added_phys
    23.8c per culling
    38.6c per t1bleeddmg
    62.7c per t1atkspd
    536.5c per t1phys
    536.5c per t1hybrid
Harvest Speed Reforge
    30.9c per hit
    ∞ per essence_added_phys
    206.1c per culling
    252.4c per t1bleeddmg
    41.5c per t1atkspd
    6184.3c per t1phys
    4122.9c per t1hybrid
Harvest Attack Reforge
    14.8c per hit
    ∞ per essence_added_phys


In [43]:
# Average modifiers on rare rolls
modifiers_from_coe_rares = [ # These may be biased but easier to write down; replace later if this gives weird results
    (1,3), (3,3), (3,1), (2,2), (1,3), (1,3),
    (2,2), (1,3), (1,3), (1,3), (1,3), (1,3),
    (3,3), (2,3), (1,3), (2,2), (2,2), (1,3),
    (2,3), (2,2), (1,3), (1,3), (2,2), (1,3),
    (1,3), (2,2), (2,3), (1,3), (2,2), (1,3),
    (3,1), (1,3), (1,3), (1,3), (2,2), (1,3),
    (3,2), (1,3), (1,3), (3,3), (3,3), (1,3),
    (1,3), (2,3), (2,3), (2,2), (1,3), (3,3),
    (1,3), (2,2), (2,2), (3,3), (1,3), (3,3),
    (2,3), (1,3), (3,3), (2,3), (3,3), (2,2),
    (3,2), (2,2), (1,3), (1,3), (1,3), (2,2),
    (2,3), (1,3), (1,3), (1,3), (3,2), (2,3),
    (2,3), (2,3), (2,3), (1,3), (1,3), (2,2),
    (1,3), (2,2), (1,3), (3,2), (2,2), (1,3),
]

Counter(modifiers_from_coe_rares).most_common()

[((1, 3), 39),
 ((2, 2), 18),
 ((2, 3), 12),
 ((3, 3), 9),
 ((3, 2), 4),
 ((3, 1), 2)]

In [ ]:
# Practical planner script
input_items_and_cost = []
goal_item = []
techniques = [] # Input item, output (cost_c, percentage list of outcomes)